## Importing relevant libraries

In [ ]:
import numpy as np
import pandas as pd
import math

from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline 

## Importing dataset - replacing nan placeholder

In [ ]:
penguins = pd.read_csv('PenguinsMV0.2.csv', index_col = 0,na_values = '?')
print(penguins.shape)
penguins.head()

In [ ]:
### Breaking dataset into X and y columns

In [ ]:
X = penguins.drop(columns=['species']).values
y = penguins['species']

### Custom Gaussin NB classifier

In [ ]:
class my_GB_3(BaseEstimator, ClassifierMixin):
    """
    Custom Gaussian Naive Bayes classifier, can handle NaN values in training dataset as well as in predict queries.
    
    """
    
    def __init__(self):
        self.dfs = {}
        self.prior_probabilities = {}
        self.std_mean_dict= {}
        self.feature_count = -1
        
    def shape_checker(self, input_array):
        
        it = iter(input_array)
        the_len = len(next(it))
        if not all(len(l) == the_len for l in it):
             raise ValueError('All list items must contain the same number of features')
    
    def fit(self, Xt, yt):
        
        
        def nan_std_mean_calculator():
            
            #have std and mean for nan cleaned transposed arrays
            for key, value in self.dfs.items():
                self.std_mean_dict[key] = {}

                for column in self.dfs[key]:
                    
                    numpystd = np.nanstd(self.dfs[key][column].values)
                    
                    numpymean = np.nanmean(self.dfs[key][column].values)
                    
                    self.std_mean_dict[key][column] = {"std" : numpystd, "mean": numpymean}

                    
        
   
        def prior_probability_generator(target_array):
            """
                Generates dictionary keys based on target values and values being the prior probabilities of of target values
                return value is said dictionary
            """
            return dict(zip(target_array.value_counts(normalize=True).index,target_array.value_counts(normalize=True).values))

        def data_splitter(full_data, target_feature):
            """
            Populates a dictionary with key values based on target feature values
            Returns said dictionary

            """

            dfs = {}

            for value in full_data[target_feature].value_counts().index.tolist():
                    dfs[value] = full_data[full_data[target_feature]== value].drop(columns=[target_feature])

            return dfs
        
        
        
        #checking the shape of the input is correct
        self.shape_checker(Xt)
        
        #Xt and Yt simply represent the feature array and target features array respectively
        self.Xt = Xt
        self.yt = yt
        
        df = pd.DataFrame(Xt)
        
        #limiting the nan values of a feature to 45% as model sees a degredation in performance beyond this threshold        
        if not all(i< 0.45 for i in df.isna().mean().to_list()):
            raise ValueError('Excessive proportion of NaN values detected. Percentage missing must be below 45%.')
            
            
        df['y'] = yt
        
        self.dfs = data_splitter(df, 'y')
        self.prior_probabilities= prior_probability_generator(df['y'])
        self.feature_count = len(Xt[0])
        
        nan_std_mean_calculator()

        
    def nan_std_mean_calculator(self):
            
        #have std and mean for nan cleaned transposed arrays
        for key, value in self.dfs.items():
            self.std_mean_dict[key] = {}

            for column in self.dfs[key]:
                values = self.dfs[key][column].values
                values = values[~np.isnan(values)]
                std = np.std(values)
                mean = np.mean(values)
                self.std_mean_dict[key][column] = {"std" : std, "mean": mean}   
                
        print(self.std_mean_dict)
        
    def predict(self, X_array):
        
        def array_input_length_checker(input_array):
            if len(X_array[0]) != self.feature_count:
                raise ValueError('Predicted item length must be the same as fit training set')
                
        
        #checking that input has same number of features as fit was trained on
        array_input_length_checker(X_array)
        
        #checking that predict input array is all of same length
        self.shape_checker(X_array)
        
         
        
        output_array = []
        
        for element in X_array:
            output_array.append(self.predict_single(element))
            
        return output_array
        

    def predict_single(self, input_value):
        
        
        def not_nan_indexes(input_list):
            """
                Returns a list of indices where values are not 
            """
            not_nan = np.argwhere(~np.isnan(input_list)).tolist()
            return [item for sublist in not_nan for item in sublist]
        
        def feature_conditional_probability_generator(x_value, feature, class_name):
            """
                Calculates the conditional probability of a feature value based on the associated x feature value passed
                Formula was sourced from assignement problem statement as requested
                return value is float of the operation result

            """
            
            exponential_component = np.exp(-((x_value - self.std_mean_dict[class_name][feature]['mean']) ** 2 / (2 * self.std_mean_dict[class_name][feature]['std'] ** 2)))
            return (1 / (np.sqrt(2 * np.pi) *  self.std_mean_dict[class_name][feature]['std']))*exponential_component
        
        
        def single_class_conditionals_generator(x_value, class_name, non_nan_list):
            """
                Generates a dictionary of the all conditional values for a single class and assignes them to a dictionary
                Probabilities for each feature of the class are calculated using the feature_conditional_probability_generator() function 
                returns said dictionary

            """

            class_conditionals = {}
            
            #here instead of iterating over all columns in the dataframe, i can simple use a for element in non_nan list
            #this will allow me to only calculate the conditionals on the non nan values present
            
            for element in non_nan_list:
                class_conditionals[element] = feature_conditional_probability_generator(x_value[element], element, class_name)

            return class_conditionals
        
        def all_class_conditional_dict_generator(x_value, non_nan_list):
            """
                Generates a dictionary of all class probability values and assigns them to a dictionary
                Probabilties for each class conditionals are calculated using the single_class_conditionals_generator() fucntion
                return value is said dictionary

            """

            all_class_conditionals = {}

            for class_name in self.dfs.keys():
                all_class_conditionals[class_name] = single_class_conditionals_generator(x_value, class_name, non_nan_features)

            return all_class_conditionals
        
        def class_probability_generator(class_name):
            """
            Generates the probability of each target class for the given predict input
            Class prior probabilities are sourced from the prior_probabilities dictionary generated from the fit() method
            Class probabilities are generated via Naive baysienne by multiplying the class prior probability by the product of all class conditionals
            return value is the class probability float value

            """
            
             
            class_prior_prob = self.prior_probabilities[class_name]
            
            #as i designed my all_class_conditional generator to only create non-nan conditionals this remains constant
            class_probability = class_prior_prob * (np.prod(list(all_class_conditionals[class_name].values())))

            return class_probability
        
        def all_class_probability_generator():
            """
            Generates a dictionary containing the class probabilties for each class of the target feature
            Class probabilties are generated from the class_probability_generator() function
            return value is said dictionary

            """

            class_probabilities = {}

            for class_name in self.dfs.keys():
                class_probabilities[class_name] = class_probability_generator(class_name)

            return class_probabilities
        
        def class_tagger(normalized_probability_dict):
            """
                Returns the key with max value from probability dict

            """
            return max(normalized_probability_dict, key=normalized_probability_dict.get)
        
        
        #input validation on predict value
        if (np.count_nonzero(np.isnan(input_value))/len(input_value) == 1.0):
            raise ValueError("Input must have at least one non-nan value")
            
        #retrieving the index of non-nan values
        non_nan_features = not_nan_indexes(input_value)
        
        #generating available conditional probability values for non-nan features
        all_class_conditionals = all_class_conditional_dict_generator(input_value, non_nan_features)
        
        #obtaining highest probability class
        predicted_probability = class_tagger(all_class_probability_generator())
        
        return predicted_probability   

In [ ]:
myGb = my_GB_3()

In [ ]:
myGb.fit(X,y)

#### Single predict tests (Adelie and Gentoo) hand picked from from dataset for proof of concept

In [ ]:
nan_test_adelie = np.array([np.nan, 18.7, 181.0, np.nan])
myGb.predict_single(nan_test_adelie)

In [ ]:
nan_test_gentoo = np.array([46.1, np.nan, 211.0, 4500.0])
myGb.predict_single(nan_test_gentoo)

***

## Exploration of imputation outputs

Comment: Here I am performing a simple exploration of the various imputation strategies.  Vizualizations of single imputation, knn and multivariate strategies are produced. Exploration of impact of imputation strategies on classifier performance is done in the next section and contrasted with performance of the custom Gaussian Bayes model which works in the presence of missing values.

While not requiered as per assignement problem statement, I opted to keep this in as I found the exploration of the imputation values themselves to be of interest. 

#### 1st - Building single imputation

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
mean_imputer.fit(X)

In [ ]:
X_mean_imputed = mean_imputer.transform(X)

#### 2nd - Building knn imputation

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
knn_imputer = KNNImputer(n_neighbors=2)

In [ ]:
X_knn_imputed = knn_imputer.fit_transform(X)

#### 3rd - Building  Iterative Imputer

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
iter_imputer = IterativeImputer(random_state=0)

In [ ]:
iter_imputer.fit(X)

In [ ]:
X_iter_imputed = iter_imputer.transform(X)

#### Quick exploration of number of differences between iterative and mean imputer values

In [ ]:
mean_difference = []
for i, (a_val, b_val) in enumerate(zip(X_mean_imputed, X_iter_imputed)):
    for i, (nested_a, nested_b) in enumerate(zip(a_val, b_val)):
        if nested_a!= nested_b:
            mean_difference.append(abs(nested_a-nested_b))
            print(f"Mean_imputed val: {nested_a}, iter_imputed val:{nested_b}, Abs difference: {abs(nested_a-nested_b)}")
            
print(f"\nNumber of differences: ", len(mean_difference), ", Mean difference: ", sum(mean_difference)/len(mean_difference))

Comment: Here we can see that iterative imputator has returned a variety of values for the missing values in penguins, let's explore this further. (Please note this was originally formatted with the 20% missing values on a single column, utilizing the updated 20% has changed the output of the cell and I have not been able to retrieve the original file).

### Vizualizing multivariate imputator outputs

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(8,6))

plt.hist(X_iter_imputed.T[3],alpha=0.5, label="iterative_imputation")
plt.hist(X_knn_imputed.T[3],alpha=0.5, label="knn_imputation")

plt.title("Comparaison of knn and iterative inputation value distributions")
plt.xlabel("Feature values")
plt.ylabel("Count")
plt.legend(loc="upper right")
plt.show

Comment: The above plot demonstrates that these two forms of imputation do indeed produce slightly different results. However these do remain broadly similar.Further investigation into the impact on model output is requiered to determine potential significance.

#### Reviewing basic dataframe staistics

In [ ]:
df_describe = pd.DataFrame(X_iter_imputed.T[3])
df_describe.describe()

In [ ]:
df_describe2 = pd.DataFrame(X_knn_imputed.T[3])
df_describe2.describe()

Comments: Interestingly the min and max values remain the sme, with minor differences in the mean, std and quartile spread. 

#### Exploration of K value on knn imputation

In [ ]:
imputers = []

plt.figure(figsize=(8,6))

for i in range(1,51,5):
    imputer = KNNImputer(n_neighbors=i)
    imputed = imputer.fit_transform(X)
    plt.hist(imputed.T[3],alpha=0.5, label=f"{i}")
    
plt.title("Comparaison of knn imputation value distributions while varying k")
plt.xlabel("Feature values")
plt.ylabel("Count")
plt.legend(loc="upper right")
plt.show

Comment: here we can see that varying the k value of the nearst neighbors output does lead to slight variations in the imputers output values as well. 

In [ ]:
## Exploration of multivariate imputation order

In [ ]:
plt.figure(figsize=(8,6))

imputation_values = []

imputation_orders =["ascending", "descending", "roman", "arabic", "random"]

for value in imputation_orders:
    imputer = IterativeImputer(random_state=0, imputation_order=value)
    imputer.fit(X)
    imputed = imputer.transform(X)
    
    imputation_values.append(imputed.T[3])
    
    plt.hist(imputed.T[3],alpha=0.5, label=f"{value}")
    
plt.title("Comparaison of multivarite imputation order on feature value distribution")
plt.xlabel("Feature values")
plt.ylabel("Count")
plt.legend(loc="upper right")
plt.show   

Comment: Much more minor differences here in terms of the values output when compared to knn.

#### Final section comments: 

Interesting how changes to parameters on both imputors does change values, naturally this would lead us to a grid search to understand how we can optimize these imputors as a way to maxmize our classifier accuracy.

***

# 2.1 Exploring the impact of imputation on classifier performance

### Exploring the approx. 40% missing values penguins dataset

In [ ]:
penguins = pd.read_csv('PenguinsMV0.4.csv', index_col = 0,na_values = '?')
print(penguins.shape)
penguins.head()
X = penguins.drop(columns=['species']).values
y = penguins['species']

#### Splitting our data into test/train sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3,
                                                    random_state=420)
X_train.shape, X_test.shape

### Single imputation pipeline

In [ ]:
#building pipeline
singleImpPipe  = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

#creating parameters dict
param_grid = {'imputer__strategy':["mean", "median", "most_frequent", "constant"]}

#creating gridsearch object
single_gs = GridSearchCV(singleImpPipe,param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)

#fitting to data
single_gs = single_gs.fit(X_train, y_train)

In [ ]:
single_gs.best_params_

In [ ]:
y_pred_gs = single_gs.predict(X_test)
print("Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_pred_gs)))
confusion_matrix(y_test, y_pred_gs)

Comment: Accuracy of 88 percent with the best imputer strategy being mean value imputation

### Knn imputation pipeline

In [ ]:
kNNpipe  = Pipeline(steps=[
    ('imputer', KNNImputer(missing_values = np.nan)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

param_grid = {'imputer__n_neighbors': [x for x in range(1,100, 3)],
              'imputer__weights': ['uniform', 'distance']}

knn_gs = GridSearchCV(kNNpipe,param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)

knn_gs = knn_gs.fit(X_train, y_train)

In [ ]:
knn_gs.best_params_

In [ ]:
y_pred_gs = knn_gs.predict(X_test)
print("Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_pred_gs)))
confusion_matrix(y_test, y_pred_gs)

Comment: Slightly higher accuracy for the knn model at 89% with an 58 k value and uniform weight.

### Multivariate pipeline

#### Defining a standard piepline

Here simple defining a standard pipeline that will act as a reference for our pipeline gridsearch


In [ ]:
multiPipe  = Pipeline(steps=[
    ('imputer', IterativeImputer()),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])


#### Defining our imputer parameters for our grid_search

Exploring the parameters for a single imputation strategu

In [ ]:
param_grid = {'imputer__imputation_order':["ascending", "descending", "roman", "arabic", "random"], 
              'imputer__initial_strategy':["mean", "median", "most_frequent", "constant"]}

In [ ]:
#### Creating out gridsearch object

In [ ]:

multi_gs = GridSearchCV(multiPipe,param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)

multi_gs = multi_gs.fit(X_train, y_train)

In [ ]:
multi_gs.best_params_

In [ ]:
y_pred_gs = multi_gs.predict(X_test)
print("Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_pred_gs)))
confusion_matrix(y_test, y_pred_gs)

Comment: Lowest accuracy of the sklearn models tested. Model is experimental so cannot judge it's final value based on this test.

#### Custom NB with explicit NaN consideration

In [ ]:
cNBpipe  = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', my_GB_3())])

In [ ]:
param_grid = {'scaler__with_mean':[True, False]}

cNB_gs = GridSearchCV(cNBpipe,param_grid, cv=10, 
                      verbose = 1, n_jobs = -1)

cNB_gs = cNB_gs.fit(X, y)

In [ ]:
y_pred_cNB_gs = cNB_gs.predict(X_test)
print("Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_pred_cNB_gs)))
confusion_matrix(y_test, y_pred_cNB_gs)

Comment: 85 percent accuracy, slightly lower than the single imputation model

### Exploring the approx. 20% missing values penguins dataset

In [ ]:
penguins = pd.read_csv('PenguinsMV0.2.csv', index_col = 0,na_values = '?')
print(penguins.shape)
penguins.head()
X = penguins.drop(columns=['species']).values
y = penguins['species']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3,
                                                    random_state=420)
X_train.shape, X_test.shape

### Single imputation

In [ ]:
#building pipeline
singleImpPipe  = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

#creating parameters dict
param_grid = {'imputer__strategy':["mean", "median", "most_frequent", "constant"]}

#creating gridsearch object
single_gs = GridSearchCV(singleImpPipe,param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)

#fitting to data
single_gs = single_gs.fit(X_train, y_train)

single_gs.best_params_

In [ ]:
y_pred_gs = single_gs.predict(X_test)
print("Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_pred_gs)))
confusion_matrix(y_test, y_pred_gs)

single_accuracy_score_20 = accuracy_score(y_test,y_pred_gs)

Comment: Higher accuracy here with less missing values

### Knn model

In [ ]:
kNNpipe  = Pipeline(steps=[
    ('imputer', KNNImputer(missing_values = np.nan)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

param_grid = {'imputer__n_neighbors': [x for x in range(1,100, 3)],
              'imputer__weights': ['uniform', 'distance']}

knn_gs = GridSearchCV(kNNpipe,param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)

knn_gs = knn_gs.fit(X_train, y_train)

knn_gs.best_params_

In [ ]:
y_pred_gs = knn_gs.predict(X_test)
print("Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_pred_gs)))
confusion_matrix(y_test, y_pred_gs)

knn_accuracy_score_20 = accuracy_score(y_test,y_pred_gs)

Comment: Similar accuracy to the single imputation model model

### Multivariate pipeline

In [ ]:
multiPipe  = Pipeline(steps=[
    ('imputer', IterativeImputer()),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

param_grid = {'imputer__imputation_order':["ascending", "descending", "roman", "arabic", "random"], 
              'imputer__initial_strategy':["mean", "median", "most_frequent", "constant"]}


multi_gs = GridSearchCV(multiPipe,param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)

multi_gs = multi_gs.fit(X_train, y_train)

multi_gs.best_params_

In [ ]:
y_pred_gs = multi_gs.predict(X_test)
print("Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_pred_gs)))
confusion_matrix(y_test, y_pred_gs)

iter_accuracy_score_20 = accuracy_score(y_test,y_pred_gs)

Comment: Interestingly, here the multivariate pipeline has outperformed the previous two models

In [ ]:
cNBpipe  = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', my_GB_3())])

param_grid = {'scaler__with_mean':[True, False]}

cNB_gs = GridSearchCV(cNBpipe,param_grid, cv=10, 
                      verbose = 1, n_jobs = -1)

cNB_gs = cNB_gs.fit(X, y)

In [ ]:
y_pred_cNB_gs = cNB_gs.predict(X_test)
print("Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_pred_cNB_gs)))
confusion_matrix(y_test, y_pred_cNB_gs)

custom_accuracy_score_20 = accuracy_score(y_test,y_pred_cNB_gs)

Comment: Similar accuracy to the multivariate mode

#### Final section comments: 

It appears that the optimal strategy to employ to maxmimze classifier accuracy evolves as the proportion of missing data varies. For lower proportions of missing data, it appears that an iterative strategy or simply ignoring the nan values yeilds the best results, while at higher percentages of missing value simple imputation has outperformed all other models. 

This requires further investigation and discussion which are outline in the following section.


***

## Research and discussion section

Naturally, due to the existance of multiple imputation models I expected there to be a variety of underlying reasons as to why and when different models yielded such results. I opted to include the following literature quote snippets 

#### Acock, Alan C. "Working with missing values." Journal of Marriage and family 67.4 (2005): 1012-1028.

Caution should be exercised due to the bias it can lead to with regards to estimates and hypothesis creation. It is important to understand the underlying dataset one is working with to take into account why the data is missing. There are various strategies that can be employed based on why the data is missing. 

Main missing data categories:

Missing Completely at Random --> missing values are randomly distributed throughout the matrix

Missing at Random --> missing data for a variable are MAR if the likelihood of missing data on the variable is not related to the participant's score on the variable, after controlling for other variables in the study

NI missing values --> missing in ways that are neither MAR nor MCAR, but nevertheless are systematic

#### Takeaway: 
Understanding the source of dataset as well as the distribution of missing values helps in understanding how that missing dataset should be interpreted and handled

#### Kumar, S., 2020. 7 Ways to Handle Missing Values in Machine Learning. [online] Medium. Available at: <https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e> [Accessed 8 December 2021].

Many ways to handle missing data there are serious drawbacks to most missing data handling strategies. Mean imputation  does not factor covariance. For small amounts of missing data rows can be deleted but does run risk of disgarding a lot of information if studying specific subset of rows.

#### Takeaway:
Best strategy is highly contextual and should be critically appraised in terms of costs before implementation

#### COMP47350 - (Assoc Professor Georgiana Ifrim):
    
Indicated that approximately 40% of missing data is grounds for dropping a column as a best practice (will depend on target feature and other information contained in missing rows).

#### Takeaway: 

Setting my limit on missing data to 45% for any given column in the fit section of the model

#### Vizualization the distribution of missing data in our sets

In [ ]:
import seaborn as sns

In [ ]:
df = pd.read_csv('PenguinsMV0.2.csv', index_col = 0,na_values = '?')
sns.heatmap(df.isnull(), cbar=False)

In [ ]:
df = pd.read_csv('PenguinsMV0.4.csv', index_col = 0,na_values = '?')
sns.heatmap(df.isnull(), cbar=False)